In [2]:
import pandas as pd
from io import StringIO
import re

In [3]:
def extractParteien(html: str) -> list[list[str]]:
    html = html.replace('\n', '')
    html = html.replace('\t', '')
    html = html.replace('</tr>', '')
    html = html.replace('</td>', '')
    html = html.replace('<br>', ' ')
    html = html.replace('&nbsp;', '')
    html = html.split('<tr>')[1:]
    html = [partei.split('<td>')[2:] for partei in html]
    return html

In [1]:
# source folder
kerg2_2021_csv = r'..\resources\electionData\sourceCSV\2021\kerg2.csv'
kerg2_2017_csv = r'..\resources\electionData\sourceCSV\2017\kerg2.csv'

bevoelkerungszahlen_2017_csv = r'..\resources\electionData\sourceCSV\2017\bevoelkerungszahlen_2017.csv'
bevoelkerungszahlen_2021_csv = r'..\resources\electionData\sourceCSV\2021\bevoelkerungszahlen_2021.csv'

parteien_2021_mitLandesliste_txt = r'..\resources\electionData\sourceCSV\2021\parteien_2021_mitLandesliste.txt'
parteien_2021_ohneLandesliste_txt = r'..\resources\electionData\sourceCSV\2021\parteien_2021_ohneLandesliste.txt'
parteien_2017_mitLandesliste_txt = r'..\resources\electionData\sourceCSV\2017\parteien_2017_mitLandesliste.txt'
parteien_2017_ohneLandesliste_txt = r'..\resources\electionData\sourceCSV\2017\parteien_2017_ohneLandesliste.txt'

kandidaturen_2021_csv = r'..\resources\electionData\sourceCSV\2021\kandidaturen_2021.csv'
kandidaturen_2017_csv = r'..\resources\electionData\sourceCSV\2017\kandidaten_2017.csv'

# target folder

bundesLand_csv = r'..\resources\electionData\targetCSV\bundesland.csv'
bevoelkerung_csv = r'..\resources\electionData\targetCSV\bevoelkerung.csv'

wahlkreise_csv = r'..\resources\electionData\targetCSV\wahlkreise.csv'

parteien_2021_csv = r'..\resources\electionData\targetCSV\2021\parteien.csv'
parteien_2021_ParteiName_csv = r'..\resources\electionData\targetCSV\2021\parteien_name.csv'
parteien_2021_EB_csv = r'..\resources\electionData\targetCSV\2021\parteien_EB.csv'

parteien_2017_csv = r'..\resources\electionData\targetCSV\2017\parteien.csv'
parteien_2017_ParteiName_csv = r'..\resources\electionData\targetCSV\2017\parteien_name.csv'

direktmandate_2021_csv = r'..\resources\electionData\targetCSV\2021\direktmandate.csv'
landesliste_2021_csv = r'..\resources\electionData\targetCSV\2021\landesliste.csv'

kandidaten_2017_csv = r'..\resources\electionData\targetCSV\2017\kandidaten.csv'

erststimme_aggr_2021_csv = r'..\resources\electionData\targetCSV\2021\erststimmeAggr.csv'
zweitstimme_aggr_2021_csv = r'..\resources\electionData\targetCSV\2021\zweitstimmeAggr.csv'

erststimme_aggr_2017_csv = r'..\resources\electionData\targetCSV\2017\erststimmeAggr.csv'
zweitstimme_aggr_2017_csv = r'..\resources\electionData\targetCSV\2017\zweitstimmeAggr.csv'

stimmen_csv = r'..\resources\electionData\targetCSV\stimmen.csv'


In [4]:
# Bundesländer

kerg2 = pd.read_csv(kerg2_2021_csv, sep=';', skiprows=9)

kerg2 = kerg2.loc[kerg2['Gebietsart'] == 'Land'] # filter for bundesland only
kerg2 = kerg2.filter(items=['Gebietsnummer', 'Gebietsname']) # drop unecessary columns
kerg2 = kerg2.drop_duplicates(subset='Gebietsnummer') 
kerg2 = kerg2.sort_values(by = 'Gebietsnummer')
kerg2 = kerg2.rename(columns={'Gebietsnummer':'id', 'Gebietsname': 'name'})

kerg2.to_csv(bundesLand_csv, index=False, sep=';')

In [5]:
# Bevölkerung

bevoelkerung_2021 = pd.read_csv(bevoelkerungszahlen_2021_csv, sep=';')
bevoelkerung_2021['jahr'] = 2021

bevoelkerung_2017 = pd.read_csv(bevoelkerungszahlen_2017_csv, sep=';')
bevoelkerung_2017['jahr'] = 2017

bevoelkerung = pd.concat([bevoelkerung_2017, bevoelkerung_2021])
bevoelkerung = bevoelkerung.rename(columns={'name':'bundesland'})

bevoelkerung.to_csv(bevoelkerung_csv, index=False, sep=';')



In [6]:
# wahlkreise

kerg2 = pd.read_csv(kerg2_2021_csv, sep=';', skiprows=9)

kerg2 = kerg2.loc[kerg2['Gebietsart'] == 'Wahlkreis'] # filter for wahlkreis only
kerg2 = kerg2.filter(items=['Gebietsnummer', 'Gebietsname', 'UegGebietsnummer', 'UegGebietsnummer']) # drop unecessary columns
kerg2['UegGebietsnummer'] = kerg2['UegGebietsnummer'].astype('int')
kerg2 = kerg2.drop_duplicates(subset='Gebietsnummer') 
kerg2 = kerg2.sort_values(by = 'Gebietsnummer')
kerg2 = kerg2.rename(columns={'Gebietsnummer':'id', 'Gebietsname': 'name', 'UegGebietsnummer': 'bundesland_id'})

kerg2.to_csv(wahlkreise_csv, index=False, sep=';')

In [7]:
# Parteien/ EB 2021 kerg2 kurzbezeichnugnen

parteien_2021 = pd.read_csv(kerg2_2021_csv, sep=';', skiprows=9)
parteien_2021 = parteien_2021.loc[parteien_2021['Gruppenart'] != 'System-Gruppe']
parteien_2021 = parteien_2021.loc[parteien_2021['Gebietsart'] == 'Wahlkreis']
parteien_2021['isEinzelbewerber'] = parteien_2021['Gruppenart'].map(lambda art: art == 'Einzelbewerber/Wählergruppe')
parteien_2021['wahlkreis_id'] = parteien_2021.apply(lambda row: row.Gebietsnummer if row.Gruppenart == 'Einzelbewerber/Wählergruppe' else -1, axis=1)
parteien_2021 = parteien_2021.filter(items=['Gruppenname', 'wahlkreis_id', 'isEinzelbewerber'])
parteien_2021 = parteien_2021.drop_duplicates()
parteien_2021 = parteien_2021.rename(columns={'Gruppenname': 'kurzbezeichnung'})

parteien_2021.to_csv(parteien_2021_csv, sep=';', index=False)

In [8]:
# Parteien 2021 LangName 

# https://www.bundeswahlleiterin.de/bundestagswahlen/2021/wahlbewerber.html
with open(parteien_2021_mitLandesliste_txt, mode ='r') as file1, open(parteien_2021_ohneLandesliste_txt, mode ='r') as file2:
    html = ''.join(file1.readlines() + file2.readlines())
    parteien = extractParteien(html)
    parteien_2021_lang = pd.DataFrame(parteien, columns=['kurzbezeichnung', 'name', 'Zusatzbezeichnung'])
    parteien_2021_lang.loc[parteien_2021_lang['Zusatzbezeichnung'] == '–', 'Zusatzbezeichnung'] = ''
    parteien_2021_lang.to_csv(parteien_2021_ParteiName_csv, sep=';', index=False)


In [9]:
# EB 2021 Details

einzelBewerber_2021 = pd.read_csv(kandidaturen_2021_csv, sep= ';')
einzelBewerber_2021 = einzelBewerber_2021.filter(items=['Gruppenname', 'Gebietsnummer', 'GruppennameLang'])
einzelBewerber_2021 = einzelBewerber_2021.rename(columns={'Gruppenname':'kurzbezeichnung', 'Gebietsnummer': 'wahlkreis_id'})

einzelBewerber_2021['dropDuplicatePartei'] = einzelBewerber_2021.apply(lambda row: row.name if row.kurzbezeichnung.startswith('EB:') else -1, axis=1)
einzelBewerber_2021 = einzelBewerber_2021.drop_duplicates(subset=['kurzbezeichnung', 'dropDuplicatePartei'])
einzelBewerber_2021 = einzelBewerber_2021.filter(items=['kurzbezeichnung', 'wahlkreis_id', 'GruppennameLang'])

einzelBewerber_2021.to_csv(parteien_2021_EB_csv, sep=';', index=False)

In [10]:
# Parteien/ EB 2017 kerg2 kurzbezeichnugnen

parteien_2017 = pd.read_csv(kerg2_2017_csv, sep=';', skiprows=9)
parteien_2017 = parteien_2017.loc[parteien_2017['Gruppenart'] != 'System-Gruppe']
parteien_2017 = parteien_2017.loc[parteien_2017['Gebietsart'] == 'Wahlkreis']
parteien_2017['isEinzelbewerber'] = parteien_2017['Gruppenart'].map(lambda art: art == 'Einzelbewerber/Wählergruppe')
parteien_2017['wahlkreis_id'] = parteien_2017.apply(lambda row: row.Gebietsnummer if row.Gruppenart == 'Einzelbewerber/Wählergruppe' else -1, axis=1)
parteien_2017 = parteien_2017.filter(items=['Gruppenname', 'wahlkreis_id', 'isEinzelbewerber'])
parteien_2017 = parteien_2017.drop_duplicates()
parteien_2021 = parteien_2021.rename(columns={'Gruppenname': 'kurzbezeichnung'})

parteien_2017.to_csv(parteien_2017_csv, sep=';', index=False)

In [11]:
# Parteien 2017 LangName 

# https://www.bundeswahlleiterin.de/bundestagswahlen/2017/wahlbewerber.html
with open(parteien_2017_mitLandesliste_txt, mode ='r') as file1,  open(parteien_2017_ohneLandesliste_txt, mode ='r') as file2:
    html = ''.join(file1.readlines()+ file2.readlines())
    parteien = extractParteien(html)
    parteien_2017_lang = pd.DataFrame(parteien, columns=['kurzbezeichnung', 'name', 'Zusatzbezeichnung'])
    parteien_2017_lang.loc[parteien_2017_lang['name'] == 'Allianz Deutscher Demokraten', 'kurzbezeichnung'] = 'AD-Demokraten' # found in kerg2 2017
    parteien_2017_lang.loc[parteien_2017_lang['name'] == 'Neue Liberale – Die Sozialliberalen', 'kurzbezeichnung'] = 'Neue Liberale' # found in kerg2 2017

    parteien_2017_lang.to_csv(parteien_2017_ParteiName_csv, sep=';', index=False)


In [36]:
# direktmandate 2021

direktmandate_2021 = pd.read_csv(kandidaturen_2021_csv, sep= ';')

direktmandate_2021 = direktmandate_2021.loc[direktmandate_2021['Gebietsart'] == 'Wahlkreis']
direktmandate_2021 = direktmandate_2021.filter(items=['Vornamen', 'Nachname', 'Geburtsjahr', 'Gruppenname', 'Gebietsname'])
direktmandate_2021 = direktmandate_2021.rename(columns={'Gruppenname': 'partei', 'Gebietsname' : 'wahlkreis'})
direktmandate_2021['jahr'] = 2021
direktmandate_2021.insert(loc=5, column='bundesland', value=-1)
direktmandate_2021.insert(loc=6, column='listenplatz', value=-1)


direktmandate_2021.to_csv(direktmandate_2021_csv, sep=';', index=False)

In [34]:
# landeslisten 2021

landesliste_2021 = pd.read_csv(kandidaturen_2021_csv, sep= ';')

landesliste_2021 = landesliste_2021.loc[landesliste_2021['Gebietsart'] == 'Land']
landesliste_2021 = landesliste_2021.filter(items=['Vornamen', 'Nachname', 'Geburtsjahr', 'Gruppenname', 'Gebietsname', 'Listenplatz'])
landesliste_2021 = landesliste_2021.rename(columns={'Gruppenname': 'partei', 'Gebietsname' : 'bundesland'})
landesliste_2021['jahr'] = 2021
landesliste_2021['Listenplatz'] = landesliste_2021['Listenplatz'].astype('int')
landesliste_2021.insert(loc=4, column='wahlkreis', value=-1)

landesliste_2021.to_csv(landesliste_2021_csv, sep=';', index=False)

In [ ]:
# kandidaturen 2017

with open(kandidaturen_2017_csv) as file:
    csv = ''.join(file.readlines())
    csv = csv.replace('\n;;;', '')

kandidaturen_2017 = pd.read_csv(StringIO(csv), sep= ';')

kandidaturen_2017['vorname'] = kandidaturen_2017['Name, Vorname(n)'].map(lambda s: s.split(', ')[1])
kandidaturen_2017['nachname'] = kandidaturen_2017['Name, Vorname(n)'].map(lambda s: s.split(', ')[0])
kandidaturen_2017['wahlkreis_id'] = kandidaturen_2017['kandidiert im'].map(lambda s: int(next(iter(re.findall('Wahlkreis ([\\d]*)', s)), -1)))
kandidaturen_2017['bundesland'] = kandidaturen_2017['kandidiert im'].map(lambda s: next(iter(re.findall('Land (.*?) \\(Platz', s)), -1))
kandidaturen_2017['listenplatz'] = kandidaturen_2017['kandidiert im'].map(lambda s: int(next(iter(re.findall('Land.*?\\(Platz\\xa0([\\d]*)', s)), -1)))
kandidaturen_2017['jahr'] = 2017
kandidaturen_2017 = kandidaturen_2017.rename(columns={'Geburts_jahr': 'geburtsjahr'})
kandidaturen_2017 = kandidaturen_2017.filter(items=['vorname', 'nachname', 'geburtsjahr', 'Partei', 'wahlkreis_id', 'bundesland', 'listenplatz', 'jahr'])
kandidaturen_2017.to_csv(kandidaten_2017_csv, sep=';', index=False)

In [ ]:
# zweitstimme aggr 2021

kerg2_2021 = pd.read_csv(kerg2_2021_csv, sep=';', skiprows=9)
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Gebietsart'] == 'Wahlkreis']
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Stimme'] == 2]
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Gruppenart'] == 'Partei']
kerg2_2021 = kerg2_2021.filter(items=['Gruppenname', 'Gebietsnummer', 'Anzahl'])
kerg2_2021.insert(loc=2, column='jahr', value=2021)
kerg2_2021['Anzahl'] = kerg2_2021['Anzahl'].map(lambda x: int(x) if not pd.isna(x) else 0)

kerg2_2021 = kerg2_2021.rename(columns={'Gruppenname':'kurzbezeichnung', 'Gebietsnummer': 'wahlkreis_id', 'Anzahl': 'anzahl'})

kerg2_2021.to_csv(zweitstimme_aggr_2021_csv, sep=';', index=False)


In [ ]:
# zweitstimme aggr 2017

kerg2_2017 = pd.read_csv(kerg2_2017_csv, sep=';', skiprows=9) 
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Gebietsart'] == 'Wahlkreis']
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Stimme'] == 2]
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Gruppenart'] == 'Partei']
kerg2_2017 = kerg2_2017.filter(items=['Gruppenname', 'Gebietsnummer', 'Anzahl'])
kerg2_2017.insert(loc=2, column='jahr', value=2017)
kerg2_2017['Anzahl'] = kerg2_2017['Anzahl'].map(lambda x: int(x) if not pd.isna(x) else 0)

kerg2_2017 = kerg2_2017.rename(columns={'Gruppenname':'kurzbezeichnung', 'Gebietsnummer': 'wahlkreis_id', 'Anzahl': 'anzahl'})

kerg2_2017.to_csv(zweitstimme_aggr_2017_csv, sep=';', index=False)


In [ ]:
# erstimme aggr 2021

kerg2_2021 = pd.read_csv(kerg2_2021_csv, sep=';', skiprows=9)
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Gebietsart'] == 'Wahlkreis']
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Stimme'] == 1]
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Gruppenart'] != 'System-Gruppe']
kerg2_2021 = kerg2_2021.filter(items=['Gruppenname', 'Gebietsnummer', 'Anzahl'])
kerg2_2021.insert(loc=2, column='jahr', value=2021)
kerg2_2021['Anzahl'] = kerg2_2021['Anzahl'].map(lambda x: int(x) if not pd.isna(x) else 0)

kerg2_2017 = kerg2_2017.rename(columns={'Gruppenname':'kurzbezeichnung', 'Gebietsnummer': 'wahlkreis_id', 'Anzahl': 'anzahl'})

kerg2_2021.to_csv(erststimme_aggr_2021_csv, sep=';', index=False)


In [ ]:
# erstimme aggr 2017

kerg2_2017 = pd.read_csv(kerg2_2017_csv, sep=';', skiprows=9)
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Gebietsart'] == 'Wahlkreis']
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Stimme'] == 1]
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Gruppenart'] != 'System-Gruppe']
kerg2_2017 = kerg2_2017.filter(items=['Gruppenname', 'Gebietsnummer', 'Anzahl'])
kerg2_2017.insert(loc=2, column='jahr', value=2017)
kerg2_2017['Anzahl'] = kerg2_2017['Anzahl'].map(lambda x: int(x) if not pd.isna(x) else 0)

kerg2_2017 = kerg2_2017.rename(columns={'Gruppenname':'kurzbezeichnung', 'Gebietsnummer': 'wahlkreis_id', 'Anzahl': 'anzahl'})

kerg2_2017.to_csv(erststimme_aggr_2017_csv, sep=';', index=False)
